### 본 코드는 사이킷런(sklearn)으로 수행하였으며, K-fold(교차검증) 방식과 2가지 방식의 모델(Multi-layer perceptron과 LinearRegression)으로 수행하였습니다.

In [1]:
import pandas as pd
import numpy as np

# Multi-layer perceptron 모델 구성
from sklearn.neural_network import MLPRegressor

# LinearRegression 모델 구성
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor

from sklearn.linear_model import PassiveAggressiveRegressor


from sklearn.linear_model import LogisticRegression

# K-fold
from sklearn.model_selection import KFold

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

np.random.seed(0)

### 데이터 불러오기
#### csv 파일 읽어들이기

In [2]:
train_df = pd.read_csv('./dataset/iris_train.csv')
train_df

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,0,setosa,4.4,1.4,2.9,0.2
1,1,versicolor,6.4,4.5,3.2,1.5
2,2,virginica,6.2,4.8,2.8,1.8
3,3,virginica,7.2,6.1,3.6,2.5
4,4,setosa,4.9,1.4,3.0,0.2
...,...,...,...,...,...,...
70,70,versicolor,6.5,4.6,2.8,1.5
71,71,versicolor,5.6,3.6,2.9,1.3
72,72,versicolor,6.2,4.5,2.2,1.5
73,73,versicolor,4.9,3.3,2.4,1.0


### 학습 데이터 전처리 & 학습 데이터/결과 데이터 분류

In [3]:
train_df['species'] = train_df['species'].map({'setosa':0, 'versicolor': 1, 'virginica': 2})

X, Y = train_df.iloc[:,1:4] , train_df.iloc[:, 4:]

In [5]:
Y1 = Y['sepal width (cm)']
Y2 = Y['petal width (cm)']

#### 본 경진대회는 3개의 특징(species, sepal length (cm), petal length)를 가지고 sepal width (cm)와 petal width (cm)를 예측하는 문제입니다.

#### 예측하고자 하는 sepal width (cm)와 petal width (cm)의 경우, 연속적인 값을 나타내므로, 연속값을 출력하는 회귀(Regression) 모델을 구성하였습니다.

#### 학습 데이터를 보니 75개의 양으로, 데이터 양이 적다 생각하여 K-fold 기법을 고려하였습니다.
#### K-fold란 학습 데이터를 K개로 분할하여 K-1 개로 학습하고 나머지 갯수로 평가하는 방법입니다.

In [6]:
from IPython.display import Image
Image(url='https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/K-fold_cross_validation_EN.svg/521px-K-fold_cross_validation_EN.svg.png')

#### 본 코드는 K-fold에서 5개의 데이터를 분할하였으며
#### 2가지의 모델을 가지고 실험 (Multi-layer perceptron model 과 LinearRegression)하였습니다.
#### K-fold와 동일한 갯수의 모델을 설계

In [8]:
def MAE(true, pred):
    score = np.mean(np.abs(true-pred))
    return score


model_arr = []
MAE_error_record = []

split_count = 5

kf = KFold(n_splits= split_count)
kf.get_n_splits(X)

for train_index, test_index in kf.split(X):

    x_train, x_test = X.values[train_index], X.values[test_index]
    y_train, y_test = Y1.values[train_index], Y1.values[test_index]
    
    # multi-layer perceptron model. 으로 해봤을때 성능 확인
    # created_model = MLPRegressor(solver='lbfgs').fit(x_train, y_train)

    # LinearRegression 으로 확인
    # created_model = LinearRegression().fit(x_train, y_train)

    # created_model = HuberRegressor().fit(x_train, y_train)
    
    created_model = PassiveAggressiveRegressor(max_iter=100, random_state=0, tol=1e-3).fit(x_train, y_train)
    
    pred = created_model.predict(x_test)
    
    MAE_error_record.append(MAE(y_test, pred))
        
    model_arr.append(created_model)
    
    
model_arr2 = []
MAE_error_record2 = []

split_count = 5

kf = KFold(n_splits= split_count)
kf.get_n_splits(X)

for train_index, test_index in kf.split(X):

    x_train, x_test = X.values[train_index], X.values[test_index]
    y_train, y_test = Y2.values[train_index], Y2.values[test_index]
    
    # multi-layer perceptron model. 으로 해봤을때 성능 확인
    # created_model = MLPRegressor(solver='lbfgs').fit(x_train, y_train)

    # LinearRegression 으로 확인
    # created_model = LinearRegression().fit(x_train, y_train)

    # created_model = HuberRegressor().fit(x_train, y_train)
    
    created_model = PassiveAggressiveRegressor(max_iter=100, random_state=0, tol=1e-3).fit(x_train, y_train)
    
    pred = created_model.predict(x_test)
    
    MAE_error_record2.append(MAE(y_test, pred))
        
    model_arr2.append(created_model)

In [10]:
print(model_arr)
print(MAE_error_record)

[PassiveAggressiveRegressor(max_iter=100, random_state=0), PassiveAggressiveRegressor(max_iter=100, random_state=0), PassiveAggressiveRegressor(max_iter=100, random_state=0), PassiveAggressiveRegressor(max_iter=100, random_state=0), PassiveAggressiveRegressor(max_iter=100, random_state=0)]
[0.2337745547283115, 0.42165604107620747, 0.4262727299541202, 0.30841078776967684, 0.3128890903213]


In [13]:
print(model_arr2)
print(MAE_error_record2)

[PassiveAggressiveRegressor(max_iter=100, random_state=0), PassiveAggressiveRegressor(max_iter=100, random_state=0), PassiveAggressiveRegressor(max_iter=100, random_state=0), PassiveAggressiveRegressor(max_iter=100, random_state=0), PassiveAggressiveRegressor(max_iter=100, random_state=0)]
[0.20563434261820926, 0.14526359750794024, 0.1779101467749781, 0.13300292049515752, 0.13519832578220944]


### K-fold의 분류 수 만큼, 모델을 생성 한 뒤   모델 중 가장 MSE Error 값이 낮은 모델을 선정

In [12]:
find_model_num = np.where(MAE_error_record == np.min(MAE_error_record))[0][0]
print(f"Find model Num :  {find_model_num} ")

final_model = model_arr[find_model_num]
print(MAE_error_record[find_model_num])

Find model Num :  0 
0.2337745547283115


In [14]:
find_model_num2 = np.where(MAE_error_record2 == np.min(MAE_error_record2))[0][0]
print(f"Find model Num :  {find_model_num} ")

final_model2 = model_arr2[find_model_num2]
print(MAE_error_record2[find_model_num2])

Find model Num :  0 
0.13300292049515752


### test 결과 값 추정

In [15]:
result_df = pd.read_csv('./dataset/iris_test.csv')
result_df['species'] = result_df['species'].map({'setosa':0, 'versicolor': 1, 'virginica': 2})

x_result = result_df.iloc[:,1:4]

y_result = final_model.predict(x_result)
y_result2 = final_model2.predict(x_result)

In [16]:
submission = pd.read_csv('./dataset/sample_submission.csv')
submission['sepal width (cm)'] = y_result
submission['petal width (cm)'] = y_result

### submission 파일이 제대로 들어갔는지 확인

In [17]:
submission

,id,sepal width (cm),petal width (cm)
0,0,3.501687,3.501687
1,1,3.797468,3.797468
2,2,3.515004,3.515004
3,3,3.205906,3.205906
4,4,2.857682,2.857682
...,...,...,...
70,70,2.935554,2.935554
71,71,2.963391,2.963391
72,72,2.733800,2.733800
73,73,2.860904,2.860904


In [18]:
submission.to_csv("result.csv", index=False)

## 추가, 본 코드에서는 MAE 값이 가장 작은 모델을 선정하였지만. 모든 모델의 predict 값을 낸 뒤 평균을 내는 방법도 있습니다.### 

In [ ]:
total_result = np.zeros_like(y_result)

for each_model in model_arr:
    
    each_result = each_model.predict(x_result)
    
    total_result += each_result
    
total_result /= split_count

In [ ]:
submission2 = pd.read_csv('./dataset/sample_submission.csv')
submission2['sepal width (cm)'] = y_result[:,0]
submission2['petal width (cm)'] = y_result[:,1]

submission2.to_csv("result2.csv", index=False)